In [ ]:
import folium
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
from math import radians, cos, sin, asin, sqrt

# Define the Haversine formula function to calculate distance on a sphere
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees) in kilometers.
    """
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371 # Radius of earth in kilometers
    return c * r

In [ ]:
# Initialize FDSN clients
client_global = Client("IRIS")
client_eu = Client("EMSC")

# Define the event time (Alenquer, Portugal M4.1 on Feb 19, 2026)
eq_time = UTCDateTime("2026-02-19T12:14:00")

print("1. Fetching Metadata & Coordinates...")
# Fetch earthquake metadata from EMSC
catalog = client_eu.get_events(starttime=eq_time - 120, endtime=eq_time + 120, minmagnitude=4.0)
eq_lat = catalog[0].origins[0].latitude
eq_lon = catalog[0].origins[0].longitude
print(f"   Epicenter found at: {eq_lat}, {eq_lon}")

# Fetch station metadata from IRIS
inventory = client_global.get_stations(network="IU", station="PAB")
sta_lat = inventory[0][0].latitude
sta_lon = inventory[0][0].longitude
print(f"   Station found at: {sta_lat}, {sta_lon}")

# --- Perform Calculations ---
distance_km = haversine_distance(eq_lat, eq_lon, sta_lat, sta_lon)

# Estimate P-wave travel time based on average crustal speed of ~7 km/s
estimated_travel_time = distance_km / 7.0

print(f"\nResults:")
print(f"The earthquake occurred {distance_km:.2f} km away from the station.")
print(f"The primary seismic waves took approximately {estimated_travel_time:.1f} seconds to arrive.")

print("\n2. Downloading seismic waveform data...")
# Fetch 20 minutes of data from Station PAB
st = client_global.get_waveforms(
    network="IU", station="PAB", location="00", channel="BHZ",
    starttime=eq_time, endtime=eq_time + 1200
)

# Apply a bandpass filter (0.5 - 5.0 Hz) for local quakes
st.filter("bandpass", freqmin=0.5, freqmax=5.0)

# Plot the waveform (Standard ObsPy plot)
st.plot(type="relative", color="red", title="Filtered Seismic Data (IU.PAB, Spain)")

In [ ]:
print("Generating enhanced map...")

# Create the Folium map centered between the two points
m = folium.Map(location=[(eq_lat + sta_lat) / 2, (eq_lon + sta_lon) / 2], zoom_start=7)

# --- Add Visual Expansion Rings ---
# Add three concentric circles to simulate wave propagation expanding from epicenter
# Radii defined in meters: 20km, 50km, 100km
expansion_radii = [20000, 50000, 100000]
for radius in expansion_radii:
    folium.Circle(
        location=[eq_lat, eq_lon],
        radius=radius,
        color="crimson",      # Border color
        weight=2,             # Border thickness
        fill=True,            # Fill the circle
        fill_opacity=0.1      # Make fill semi-transparent
    ).add_to(m)

# Add Markers
folium.Marker(
    [eq_lat, eq_lon],
    popup="Magnitude 4.1 Epicenter (Alenquer)",
    icon=folium.Icon(color="red", icon="info-sign")
).add_to(m)

# Update station popup to include computed distance
station_popup_text = f"Station IU.PAB (Spain)<br>Distance: {distance_km:.1f} km"
folium.Marker(
    [sta_lat, sta_lon],
    popup=station_popup_text,
    icon=folium.Icon(color="blue", icon="flag")
).add_to(m)

# Draw the wave path line
folium.PolyLine([(eq_lat, eq_lon), (sta_lat, sta_lon)], color="black", dash_array="5, 5", weight=3).add_to(m)

# Display map
m